In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
from math import pi
from builtins import range
import numpy as np

In [2]:
def sphereloss(inputs,label,classes,fraction = 1, scope='Logits',reuse=None,m =4,eplion = 1e-8):
    """
    inputs tensor shape=[batch,features_num]
    labels tensor shape=[batch] each unit belong num_outputs
    
    """
    inputs_shape = inputs.get_shape().as_list()
    with tf.variable_scope(name_or_scope=scope):
        weight = tf.Variable(initial_value=tf.random_normal((classes,inputs_shape[1])) * tf.sqrt(2 / inputs_shape[1]),dtype=tf.float32,name='weight') # shaep =classes, features,
        print("weight shape = ",weight.get_shape().as_list())
    
    weight_unit = tf.nn.l2_normalize(weight,dim=1)
    print("weight_unit shape = ",weight_unit.get_shape().as_list())
    
    inputs_mo = tf.sqrt(tf.reduce_sum(tf.square(inputs),axis=1)+eplion) #shape=[batch
    print("inputs_mo shape = ",inputs_mo.get_shape().as_list())
    
    inputs_unit = tf.nn.l2_normalize(inputs,dim=1)  #shape = [batch,features_num]
    print("inputs_unit shape = ",inputs_unit.get_shape().as_list())
    
    logits = tf.matmul(inputs,tf.transpose(weight_unit)) #shape = [batch,classes] x * w_unit
    print("logits shape = ",logits.get_shape().as_list())
    
    weight_unit_batch = tf.gather(weight_unit,label) # shaep =batch,features_num,
    print("weight_unit_batch shape = ",weight_unit_batch.get_shape().as_list())
    
    logits_inputs = tf.reduce_sum(tf.multiply(inputs,weight_unit_batch),axis=1) # shaep =batch,
    
    print("logits_inputs shape = ",logits_inputs.get_shape().as_list())
    
    
    cos_theta = tf.reduce_sum(tf.multiply(inputs_unit,weight_unit_batch),axis=1) # shaep =batch,
    print("cos_theta shape = ",cos_theta.get_shape().as_list())
    
    cos_theta_square = tf.square(cos_theta)
    cos_theta_biq = tf.pow(cos_theta,4)
    sign0 = tf.sign(cos_theta)
    sign2 = tf.sign(2 * cos_theta_square-1)
    sign3 = tf.multiply(sign2,sign0)
    sign4 = 2 * sign0 +sign3 -3
    cos_far_theta = sign3 * (8 * cos_theta_biq - 8 * cos_theta_square + 1) + sign4
    print("cos_far_theta  = ",cos_far_theta.get_shape().as_list())
    
    logit_ii = tf.multiply(cos_far_theta,inputs_mo)#shape = batch    
    print("logit_ii shape = ",logit_ii.get_shape().as_list())
    
    
        
    index = tf.constant(list(range(0, inputs_shape[0])), tf.int32)
    index_labels = tf.stack([index, label], axis = 1)
    index_logits =  tf.scatter_nd(index_labels,tf.subtract(logit_ii,logits_inputs),logits.get_shape())
    print("index_logits shape = ",logit_ii.get_shape().as_list())
    
    logits_final  = tf.add(logits,index_logits)
    logits_final = fraction * logits_final + (1 - fraction) * logits
    
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label,logits=logits_final))
    
    return logits_final,loss
    
    
    
    

In [3]:
np.random.seed(100)
batch = 16
feating = 128
classess = 10000
# inputsinputs =  [[-1.0,1.0],[1.0,1.0],[1.0,1.0]]

inputsinputs = np.random.rand(batch,feating)  
print(inputsinputs)
inputslables = np.random.randint(0,classess,batch)
print(inputslables)

[[ 0.54340494  0.27836939  0.42451759 ...,  0.49216697  0.40288033
   0.3542983 ]
 [ 0.50061432  0.44517663  0.09043279 ...,  0.91975536  0.84960733
   0.25446654]
 [ 0.87755554  0.43513019  0.72949434 ...,  0.34675413  0.1095646   0.378327  ]
 ..., 
 [ 0.37716757  0.75750263  0.29912515 ...,  0.53643677  0.63122505
   0.17644686]
 [ 0.5396841   0.55346366  0.30105263 ...,  0.97211104  0.51628182
   0.44451879]
 [ 0.29864351  0.8312721   0.28519946 ...,  0.09984372  0.71015638
   0.37341943]]
[ 282 9194 3716 2977 9688 3074  487 6080  467 4974 1458 4028 9966 5131  583
 2955]


In [4]:
inputs_place = tf.placeholder(dtype=tf.float32,shape=(batch,feating),name='inputs')
labels_place = tf.placeholder(dtype=tf.int32,shape=(batch),name='labels')

In [5]:
_,loss = sphereloss(inputs_place,labels_place,classess)
optimizer = tf.train.AdamOptimizer(learning_rate=0.05)
train_op = optimizer.minimize(loss)

weight shape =  [10000, 128]
weight_unit shape =  [10000, 128]
inputs_mo shape =  [16]
inputs_unit shape =  [16, 128]
logits shape =  [16, 10000]
weight_unit_batch shape =  [16, 128]
logits_inputs shape =  [16]
cos_theta shape =  [16]
cos_far_theta  =  [16]
logit_ii shape =  [16]
index_logits shape =  [16]


In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for eopch in range(1000):    
        print(sess.run([loss,train_op],feed_dict={inputs_place:inputsinputs,labels_place:inputslables}))

[28.798058, None]
[22.013681, None]
[15.798929, None]
[13.435317, None]
[12.685493, None]
[11.346285, None]
[9.4595337, None]
[7.3966951, None]
[5.5232167, None]
[4.1032286, None]
[3.2184033, None]
[2.7606359, None]
[2.5476418, None]
[2.4361296, None]
[2.3488777, None]
[2.2575321, None]
[2.1591675, None]
[2.0600286, None]
[1.9670352, None]
[1.884537, None]
[1.8141112, None]
[1.7553788, None]
[1.7070088, None]
[1.6673062, None]
[1.6345801, None]
[1.6072876, None]
[1.5841055, None]
[1.5639385, None]
[1.5459807, None]
[1.5295949, None]
[1.5144105, None]
[1.5001791, None]
[1.4868002, None]
[1.474241, None]
[1.462525, None]
[1.4517064, None]
[1.4418349, None]
[1.432952, None]
[1.42506, None]
[1.418117, None]
[1.4120786, None]
[1.4068146, None]
[1.4022279, None]
[1.3981946, None]
[1.394609, None]
[1.3913933, None]
[1.3884966, None]
[1.3858668, None]
[1.3834978, None]
[1.3813753, None]
[1.3794972, None]
[1.377805, None]
[1.3763015, None]
[1.3749441, None]
[1.3737118, None]
[1.3725736, None]
[

[1.3604589, None]
[1.3604591, None]
[1.3604579, None]
[1.3604562, None]
[1.3604577, None]
[1.360456, None]
[1.360455, None]
[1.3604559, None]
[1.3604553, None]
[1.3604565, None]
[1.3604563, None]
[1.360455, None]
[1.360455, None]
[1.3604553, None]
[1.3604541, None]
[1.3604548, None]
[1.3604541, None]
[1.3604541, None]
[1.3604527, None]
[1.3604522, None]
[1.3604528, None]
[1.3604532, None]
[1.3604512, None]
[1.3604516, None]
[1.3604524, None]
[1.3604527, None]
[1.3604497, None]
[1.3604493, None]
[1.3604485, None]
[1.3604513, None]
[1.3604528, None]
[1.3604487, None]
[1.3604469, None]
[1.3604493, None]
[1.3604496, None]
[1.3604503, None]
[1.3604494, None]
[1.3604501, None]
[1.3604507, None]
[1.3604501, None]
[1.3604491, None]
[1.3604496, None]
[1.3604499, None]
[1.3604475, None]
[1.3604467, None]
[1.3604462, None]
[1.3604478, None]
[1.3604475, None]
[1.360445, None]
[1.3604461, None]
[1.3604487, None]
[1.3604486, None]
[1.360446, None]
[1.3604457, None]
[1.3604449, None]
[1.3604445, None

[1.3604263, None]
[1.3604246, None]
[1.3604249, None]
[1.3604234, None]
[1.360425, None]
[1.3604245, None]
[1.3604246, None]
[1.3604239, None]
[1.3604271, None]
[1.3604264, None]
[1.3604269, None]
[1.3604271, None]
[1.3604252, None]
[1.3604259, None]
[1.3604225, None]
[1.3604227, None]
[1.3604248, None]
[1.3604262, None]
[1.3604263, None]
[1.3604252, None]
[1.3604264, None]
[1.3604258, None]
[1.3604263, None]
[1.3604259, None]
[1.360425, None]
[1.3604273, None]
[1.3604286, None]
[1.360429, None]
[1.3604264, None]
[1.3604259, None]
[1.3604256, None]
[1.3604267, None]
[1.3604224, None]
[1.3604224, None]
[1.3604245, None]
[1.3604246, None]
[1.360423, None]
[1.3604246, None]
[1.3604249, None]
[1.3604242, None]
[1.3604236, None]
[1.3604255, None]
[1.360424, None]
[1.3604248, None]
[1.3604264, None]
[1.3604243, None]
[1.3604252, None]
[1.3604257, None]
[1.3604244, None]
[1.3604257, None]
[1.3604285, None]
[1.3604257, None]
[1.360424, None]
[1.360424, None]
[1.3604226, None]
[1.3604243, None]

In [ ]:
def sphereloss_mine(inputs,label,classes,weights_decay = 0.05,scope='Logits',reuse=None,m =4,eplion = 1e-12):
    
    """
    inputs tensor shape=[batch,features_num]
    labels tensor shape=[batch] each unit belong num_outputs
    
    """
    features_num = inputs.get_shape().as_list()[1]

    with tf.variable_scope(name_or_scope=scope):
        weight = tf.Variable(initial_value=tf.random_normal((classes,features_num),stddev=0.01),dtype=tf.float32,name='weight') # shaep =classes, features,
        print("weight shape = ",weight.get_shape().as_list())
        
    weight_unit = tf.nn.l2_normalize(weight,dim=1)
    print("weight_unit shape = ",weight_unit.get_shape().as_list())
    
    inputs_mo = tf.sqrt(tf.reduce_sum(tf.square(inputs),axis=1)+eplion) #shape=[batch
    print("inputs_mo shape = ",inputs_mo.get_shape().as_list())
    
    inputs_unit = tf.nn.l2_normalize(inputs,dim=1)  #shape = [batch,features_num]
    print("inputs_unit shape = ",inputs_unit.get_shape().as_list())
    
    logits = tf.matmul(inputs,tf.transpose(weight_unit)) #shape = [batch,classes] x * w_unit
    print("logits shape = ",logits.get_shape().as_list())
    
    weight_unit_batch = tf.gather(weight_unit,label) # shaep =batch,features_num,
    print("weight_unit_batch shape = ",weight_unit_batch.get_shape().as_list())
    
    logits_inputs = tf.reduce_sum(tf.multiply(inputs,weight_unit_batch),axis=1) # shaep =batch,
    
    print("logits_inputs shape = ",logits_inputs.get_shape().as_list())
    
    cos_theta = tf.reduce_sum(tf.multiply(inputs_unit,weight_unit_batch),axis=1) # shaep =batch,
    print("print shape = ",cos_theta.get_shape().as_list())
        
    k = tf.Variable(initial_value=tf.zeros_like(cos_theta),trainable=False)
    k = tf.assign(k,tf.floor_div(tf.acos(cos_theta),pi/m)+0.0)  # shaep =batch,
    print("k shape = ",k.get_shape().as_list())
    
    cos_four_theta =tf.multiply(tf.pow(cos_theta,4),8) - tf.multiply(tf.pow(cos_theta,2),8)-1 #shape = batch
    print("cos_four_theta shape = ",cos_four_theta.get_shape().as_list())
    
    cos_far_theta =tf.add(tf.multiply(tf.pow(-1.0,k) , cos_four_theta), tf.multiply(-2.0 , k)) #shape = batch
    print("cos_far_theta  = ",cos_far_theta.get_shape().as_list())
    
    
    
    logit_ii = tf.multiply(cos_far_theta,inputs_mo)#shape = batch    
    print("logit_ii shape = ",logit_ii.get_shape().as_list())
    
    logit_ii_exp = tf.exp(logit_ii)
    logits_exp = tf.reduce_sum(tf.exp(logits),axis=1) 
    logits_inputs_exp = tf.exp(logits_inputs)
    loss_exp = tf.divide (logit_ii_exp,tf.subtract(tf.add(logit_ii_exp,logits_exp ),logits_inputs_exp)) 
    #shape = batch
    print("loss_exp shape = ",loss_exp.get_shape().as_list())
    loss = -tf.reduce_mean(tf.log(loss_exp))
    print("loss shape = ",loss.get_shape().as_list())
    
#     return weight,weight_unit,weight_unit_batch,inputs,inputs_mo,inputs_unit, logits,logits_inputs,cos_theta,k,cos_four_theta,cos_far_theta,logit_ii,loss_exp,loss
#     return weight,weight_unit,weight_unit_batch,inputs_mo,inputs_unit, logits,logits_inputs,cos_theta,k,cos_four_theta,cos_far_theta,logit_ii,loss_exp, loss
    return k,loss